In [ ]:
# Installing required libraries
!pip install fastapi uvicorn pyngrok googlemaps

In [2]:
# Authentication key for Ngrok (This is only so I can host the API locally)
!ngrok config add-authtoken 2ZzAtoj8JtZVi5P0T3dxkZegVDH_413Q4uEX66XPqAS6a49qg

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
import pandas as pd
import numpy as np
import datetime
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import googlemaps
import pytz

from google_api import google_api_call

app = FastAPI()

# API Home page
@app.get('/')
async def root() :
  return {
      "Consolidated": "Go To '/con'",
      "Detailed": "Go To '/det'",
  }


# Consolidated dataset api call
@app.get('/con')
async def consolidated():
  # Temporary data for testing
  df = pd.read_csv('data.csv')

  # This will call the google api function and will create our dataset.
  # df = google_api_call('GOOGLE API KEY HERE')

  def get_formatted_datetime():
    now = datetime.datetime.now(pytz.timezone('Asia/Riyadh'))
    formatted_datetime = now.strftime("%B %dth, %Y %I:%M %p")
    return formatted_datetime

  def get_weight_average(df, brand_name):
    subset = df.loc[df['Brand'] == brand_name]
    data_points = len(subset)
    review_count = subset['Review Count'].astype(float).values
    rating = subset['Rating'].values
    weighted_sum = np.sum(review_count * rating)
    total_count = np.sum(review_count)
    weighted_average = weighted_sum / total_count

    return f'{weighted_average:.2f}', total_count, data_points

  responses = [{"Date and Time" : get_formatted_datetime()}]
  for key in df['Brand'].unique() :
    weighted_average, total_count, data_points = get_weight_average(df, key)
    results = {
        "Brand" : key,
        "Weighted Rating" : weighted_average,
        "Review Count" : total_count,
        "Locations" : data_points,
    }
    responses.append(results)

  return responses


# Detailed dataset api call
@app.get('/det')
async def detailed():
  # Temporary data for testing
  df = pd.read_csv('data.csv')

  # This will call the google api function and will create our dataset.
  # df = google_api_call('GOOGLE API KEY HERE')

  def get_formatted_datetime():
    now = datetime.datetime.now(pytz.timezone('Asia/Riyadh'))
    formatted_datetime = now.strftime("%B %dth, %Y %I:%M %p")
    return formatted_datetime

  def row_to_dict(row):
    return row.to_dict()

  # This fucntion will map the arabic names of cities, districts, and regions to their english counterparts.
  def map_func(df, mapping, column):
    df[column] = df[column].map(mapping).fillna(df[column])
    return df

  city_mapping = {'الرياض': 'Riyadh', 'الدمام':'Dammam', 'Al Dammam':'Dammam','جدة':'Jeddah', 'الخبر':'Al Khobar', 'الهفوف':'Al Hofuf', 'خميس مشيط':'Khamis Mushait',\
                  'الظهران':'Dhahran', 'أبها':'Abha', 'سيهات':'Saihat', 'مكة المكرمة':'Makkah', 'تبوك':'Tabuk', 'حائل':'Hail', 'HUBTOWNSEAST':'Hafar Al Batin', 'بريدة':'Buraydah', 'HUB TOWNS EAST':'Hafar Al Batin',\
                  'At Taif':'Taif', 'الطائف':'Taif', 'Al Baten':'Hafar Al Batin', "الهفوف‎":'Al Hofuf', 'سيهات‎':'Saihat'}

  district_mapping = {'امارة منطقة الرياض':'Riyadh Principality', 'الدمام':'Dammam Principality', 'الخبر':'Al Khobar', 'القطيف':'Al-Qatif', 'امارة منطقة مكة المكرمة':'Makkah Principality',\
                      'خميس مشيط':'Khamis Mushait', 'الاحساء':'Al Ahsa', 'بريدة':'Buraydah Principality', 'امارة منطقة عسير':'Asir Principality','جدة':'Jeddah', 'حفر الباطن':'Hafar Al-Batin',\
                      'امارة منطقة تبوك':'Tabuk Principality'}

  region_mapping = {'المنطقة الشرقية':'Eastern Province', 'منطقة الرياض':'Riyadh Province', 'مكة المكرمة':'Makkah Province', 'منطقة عسير':'Aseer Province',\
                    'منطقة تبوك':'Tabuk Province', 'منطقة القصيم':'Al Qassim Province'}

  df = map_func(df, city_mapping, 'City')
  df = map_func(df, district_mapping, 'District')
  df = map_func(df, region_mapping, 'Region')

  responses = [{"Date and Time" : get_formatted_datetime()}]
  result = df.apply(row_to_dict, axis=1)

  for i in result :
    responses.append(i)

  return responses


# This block of code is so I can run the API locally on Google Colab.
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://2ee5-34-80-146-224.ngrok-free.app


INFO:     Started server process [277]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     84.22.254.6:0 - "GET /con HTTP/1.1" 200 OK
INFO:     84.22.254.6:0 - "GET /det HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [277]
